In [1]:
# Need this for this demo to even be possible,
# turns out that jupyter already runs in its own
#  event loop which is NOT pretty to deal with
import nest_asyncio
nest_asyncio.apply()
import asyncio

%load_ext memory_profiler

In [2]:
import requests

print(requests.get("http://0.0.0.0:8080").text)
input()
print(requests.get("http://0.0.0.0:8080/items").text[:1000])
input()
print(requests.get("http://0.0.0.0:8080/items/1").text)

Hello, world

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 

In [3]:
for _ in range(70):
    print(requests.get("http://0.0.0.0:8080/items/1").text)

{"email": "cparks@example.org"}
{"email": "cparks@example.org"}
{"email": "cparks@example.org"}
{"email": "cparks@example.org"}
{"email": "cparks@example.org"}
{"email": "cparks@example.org"}
{"email": "cparks@example.org"}
{"email": "cparks@example.org"}
{"email": "cparks@example.org"}
{"email": "cparks@example.org"}
{"email": "cparks@example.org"}
{"email": "cparks@example.org"}
{"email": "cparks@example.org"}
{"email": "cparks@example.org"}
{"email": "cparks@example.org"}
{"email": "cparks@example.org"}
{"email": "cparks@example.org"}
{"email": "cparks@example.org"}
{"email": "cparks@example.org"}
{"email": "cparks@example.org"}
{"email": "cparks@example.org"}
{"email": "cparks@example.org"}
{"email": "cparks@example.org"}
{"email": "cparks@example.org"}
{"email": "cparks@example.org"}
{"email": "cparks@example.org"}
{"email": "cparks@example.org"}
{"email": "cparks@example.org"}
{"email": "cparks@example.org"}
{"email": "cparks@example.org"}
{"email": "cparks@example.org"}
{"email"

In [4]:
from aiohttp import request
import asyncio

async def req(i):
    async with request("get", f"http://0.0.0.0:8080/items/{i}") as r:
        print(await r.content.read())

asyncio.run(req(1))

b'{"email": "cparks@example.org"}'


In [5]:
from throttler import Throttler

async def req(i, t: Throttler):
    print(i, "starting")
    async with t:
        print(i, "inside throttler")
        async with request("get", f"http://0.0.0.0:8080/items/{i}") as r:
            print(i, "before request")
            resp = await r.content.read()
            print(i, "after request")
            return resp

t = Throttler(rate_limit=60, period=10.0)
results = asyncio.run(asyncio.gather(*[asyncio.create_task(req(i, t)) for i in range(70)]))

0 starting
0 inside throttler
1 starting
1 inside throttler
2 starting
2 inside throttler
3 starting
3 inside throttler
4 starting
4 inside throttler
5 starting
5 inside throttler
6 starting
6 inside throttler
7 starting
7 inside throttler
8 starting
8 inside throttler
9 starting
9 inside throttler
10 starting
10 inside throttler
11 starting
11 inside throttler
12 starting
12 inside throttler
13 starting
13 inside throttler
14 starting
14 inside throttler
15 starting
15 inside throttler
16 starting
16 inside throttler
17 starting
17 inside throttler
18 starting
18 inside throttler
19 starting
19 inside throttler
20 starting
20 inside throttler
21 starting
21 inside throttler
22 starting
22 inside throttler
23 starting
23 inside throttler
24 starting
24 inside throttler
25 starting
25 inside throttler
26 starting
26 inside throttler
27 starting
27 inside throttler
28 starting
28 inside throttler
29 starting
29 inside throttler
30 starting
30 inside throttler
31 starting
31 inside thrott

In [6]:
from tenacity import retry, stop_after_attempt, before_log

from throttler import Throttler

import logging

logging.basicConfig(stream=sys.stderr, level=logging.DEBUG)

logger = logging.getLogger(__name__)

# @retry(stop=stop_after_attempt(3), before=before_log(logger, logging.DEBUG))
@retry(stop=stop_after_attempt(3))
async def req(i, t: Throttler):
    async with t:
        async with request("get", f"http://0.0.0.0:8080/items/{i}") as r:
            resp = await r.content.read()
            r.raise_for_status()
            return resp

def run_reqs(rate_limit: int, period: float, n_reqs: int):
    t = Throttler(rate_limit=rate_limit, period=period)
    asyncio.run(asyncio.gather(*[asyncio.create_task(req(i, t)) for i in range(n_reqs)]))

def run_reqs_2(rate_limit: int, period: float, n_reqs: int):
    t = Throttler(rate_limit=rate_limit, period=period)
    tasks = [req(i, t) for i in range(n_reqs)]

async def testr(n):
    return n

def create_tasks(n):
    tasks = [asyncio.create_task(testr(i)) for i in range(n)]

def create_tasks_2(n):
    tasks = [testr(i) for i in range(n)]

In [7]:
# %memit run_reqs(10, 1.0, 1000)
%memit run_reqs_2(10, 1.0, 1000)
# %memit run_reqs_3(10, 1.0, 1000)
%memit create_tasks(1000)
%memit create_tasks(5000)
%memit create_tasks(20_000)
%memit run_reqs_2(10, 1.0, 20_000)

<string>:1: RuntimeWarning: coroutine 'req' was never awaited


peak memory: 70.57 MiB, increment: 0.30 MiB
peak memory: 73.83 MiB, increment: 3.26 MiB
peak memory: 88.30 MiB, increment: 14.40 MiB
peak memory: 145.21 MiB, increment: 56.91 MiB
peak memory: 158.63 MiB, increment: 13.32 MiB


In [8]:
import asyncio
import time

class ThrottledQueue(asyncio.Queue):
    "subclass asyncio.Queue i.e. import all behaviour"

    def __init__(self, per_second, debug=False, maxsize=0, *, loop=None, i=0):
        "Set up some extra vars and then call the original init"

        self.lock = asyncio.Lock()
        self.i = i
        self.per_second = per_second
        self.last_get = time.time() # this is the fastest way... I think?
        self.debug = debug
        super(ThrottledQueue, self).__init__(maxsize=maxsize, loop=loop)

    async def notify(self):
        """
        Signals to the queue that an item is being retried, 
        so pause any get()s by aquiring the lock and throttling before releasing
        """
        async with self.lock:
            await self._throttle()

    async def lock(self, n_seconds: int):
        async with self.lock:
            await asyncio.sleep(n_seconds)

    async def get(self):
        async with self.lock:
            await self._throttle()
            result = await super(ThrottledQueue, self).get()

            self.last_get = time.time()
            return result

    async def _throttle(self):
        elapsed = time.time() - self.last_get
        sleep_time = (1/float(self.per_second)) - elapsed
        if self.debug:
            print(self.i, '- times', f'{elapsed:.5f}', '+', f'{sleep_time:.5f}', '=', self.per_second, '- sizes', self.qsize(), f'{self.qsize() / self.maxsize:.5f}')
        await asyncio.sleep(max(0, sleep_time)) # Make sure we wait at least 0 seconds

In [9]:
import json

from collections import Counter

import sys
from aiohttp import ClientResponseError

class Sentinel: pass

async def get_all_items(q, cntr: Counter):
    async with request("get", "http://0.0.0.0:8080/items") as r:
        resp = await r.read()
        r.raise_for_status()
        cntr["success"] += 1
        for i, d in enumerate(json.loads(resp)):
            await q.put((i, d))
        await q.put((i, Sentinel))


async def handle_error(e, q):
    print(f"HANDLING ERROR: {e}")
    if e.status == 429:
        q.retry()
    elif e.status == 503:
        q.lock(10)

async def item_worker(q, idx, cntr: Counter, ostream = sys.stdout):
    while True:
        i, d = await q.get()
        if d == Sentinel:
            await q.put((i, Sentinel))
            print(f"worker {idx} exiting")
            return
        try:
            async with request("get", f"http://0.0.0.0:8080/items/{d}") as req:
                resp = await req.read()
                req.raise_for_status()
                print(f"worker {idx} response for #{i}: {resp}", file=ostream)
                cntr["success"] += 1
        except ClientResponseError as e:
            await handle_error(e, q)

def run(per_second, n_workers=10, ostream=sys.stdout, debug=False):
    cntr = Counter()
    q = ThrottledQueue(per_second=per_second, maxsize=1000, debug=debug)
    asyncio.run(
        asyncio.gather(
            asyncio.create_task(get_all_items(q, cntr)),
            *[asyncio.create_task(item_worker(q, i, cntr, ostream)) for i in range(n_workers)],
        )
    )
    return cntr

In [10]:
run(100, n_workers=10)

worker 0 response for #0: b'{"email": "gomezkelly@example.com"}'
worker 1 response for #1: b'{"email": "cparks@example.org"}'
worker 2 response for #2: b'{"email": "taylor11@example.com"}'
worker 3 response for #3: b'{"email": "greentracy@example.org"}'
worker 4 response for #4: b'{"email": "scottdavis@example.com"}'
worker 5 response for #5: b'{"email": "bjackson@example.net"}'
worker 6 response for #6: b'{"email": "lisa03@example.org"}'
worker 7 response for #7: b'{"email": "nscott@example.com"}'
worker 8 response for #8: b'{"email": "richardrobert@example.org"}'
worker 9 response for #9: b'{"email": "wromero@example.com"}'
worker 0 response for #10: b'{"email": "fordnicole@example.org"}'
worker 1 response for #11: b'{"email": "nguyensarah@example.com"}'
worker 2 response for #12: b'{"email": "patrick50@example.com"}'
worker 3 response for #13: b'{"email": "hgriffith@example.net"}'
worker 4 response for #14: b'{"email": "ilucas@example.net"}'
worker 5 response for #15: b'{"email": "e

Counter({'success': 1001})

In [11]:
with open("results.txt", "w") as ostream:
    run(per_second=100, n_workers=100, ostream=ostream, debug=True)

0 - times 0.00224 + 0.00776 = 100 - sizes 0 0.00000
0 - times 0.00045 + 0.00955 = 100 - sizes 1000 1.00000
0 - times 0.00044 + 0.00956 = 100 - sizes 999 0.99900
0 - times 0.00043 + 0.00957 = 100 - sizes 998 0.99800
0 - times 0.00033 + 0.00967 = 100 - sizes 997 0.99700
0 - times 0.00042 + 0.00958 = 100 - sizes 996 0.99600
0 - times 0.00048 + 0.00952 = 100 - sizes 995 0.99500
0 - times 0.00039 + 0.00961 = 100 - sizes 994 0.99400
0 - times 0.00034 + 0.00966 = 100 - sizes 993 0.99300
0 - times 0.00052 + 0.00948 = 100 - sizes 992 0.99200
0 - times 0.00051 + 0.00949 = 100 - sizes 991 0.99100
0 - times 0.00061 + 0.00939 = 100 - sizes 990 0.99000
0 - times 0.00040 + 0.00960 = 100 - sizes 989 0.98900
0 - times 0.00043 + 0.00957 = 100 - sizes 988 0.98800
0 - times 0.00040 + 0.00960 = 100 - sizes 987 0.98700
0 - times 0.00035 + 0.00965 = 100 - sizes 986 0.98600
0 - times 0.00039 + 0.00961 = 100 - sizes 985 0.98500
0 - times 0.00036 + 0.00964 = 100 - sizes 984 0.98400
0 - times 0.00036 + 0.00964 =

In [12]:
run(per_second=100, n_workers=10, debug=True)

0 - times 0.00097 + 0.00903 = 100 - sizes 0 0.00000
0 - times 0.00073 + 0.00927 = 100 - sizes 1000 1.00000
worker 0 response for #0: b'{"email": "gomezkelly@example.com"}'
0 - times 0.00063 + 0.00937 = 100 - sizes 999 0.99900
worker 1 response for #1: b'{"email": "cparks@example.org"}'
0 - times 0.00066 + 0.00934 = 100 - sizes 998 0.99800
worker 2 response for #2: b'{"email": "taylor11@example.com"}'
0 - times 0.00065 + 0.00935 = 100 - sizes 997 0.99700
worker 3 response for #3: b'{"email": "greentracy@example.org"}'
0 - times 0.00065 + 0.00935 = 100 - sizes 996 0.99600
worker 4 response for #4: b'{"email": "scottdavis@example.com"}'
0 - times 0.00078 + 0.00922 = 100 - sizes 995 0.99500
worker 5 response for #5: b'{"email": "bjackson@example.net"}'
0 - times 0.00076 + 0.00924 = 100 - sizes 994 0.99400
worker 6 response for #6: b'{"email": "lisa03@example.org"}'
0 - times 0.00073 + 0.00927 = 100 - sizes 993 0.99300
worker 7 response for #7: b'{"email": "nscott@example.com"}'
0 - times 0

Counter({'success': 1001})

## Fill the Queue beforehand

In [ ]:
import json

from collections import Counter

import sys
from aiohttp import ClientResponseError

class Sentinel: pass

async def get_all_items(q, cntr: Counter):
    async with request("get", "http://0.0.0.0:8080/items") as r:
        resp = await r.read()
        r.raise_for_status()
        cntr["success"] += 1
        for i, d in enumerate(json.loads(resp)):
            await q.put((i, d))
        await q.put((i, Sentinel))


async def handle_error(e, q):
    print(f"HANDLING ERROR: {e}")
    if e.status == 429:
        q.retry()
    elif e.status == 503:
        q.lock(10)

async def item_worker(q, idx, cntr: Counter, ostream = sys.stdout):
    while True:
        i, d = await q.get()
        if d == Sentinel:
            await q.put((i, Sentinel))
            print(f"worker {idx} exiting")
            return
        try:
            async with request("get", f"http://0.0.0.0:8080/items/{d}") as req:
                resp = await req.read()
                req.raise_for_status()
                print(f"worker {idx} response for #{i}: {resp}", file=ostream)
                cntr["success"] += 1
        except ClientResponseError as e:
            await handle_error(e, q)

def run(per_second, n_workers=10, ostream=sys.stdout, debug=False):
    cntr = Counter()
    q = ThrottledQueue(per_second=per_second, maxsize=1000, debug=debug)
    asyncio.run(
        asyncio.gather(
            asyncio.create_task(get_all_items(q, cntr)),
            *[asyncio.create_task(item_worker(q, i, cntr, ostream)) for i in range(n_workers)],
        )
    )
    return cntr

In [17]:
async def fill_queue(q, items):
    for i in items:
        await q.put(i)
    return q

q = asyncio.run(fill_queue(ThrottledQueue(per_second=10, maxsize=0, debug=True), list(range(1000))))

In [19]:
q.qsize(), sys.getsizeof(q), sys.getsizeof(list(range(1000)))

(1000, 48)